# AZS + Retail — Colab Demo (быстрый старт)

Этот ноутбук клонирует репозиторий, ставит зависимости, генерирует toy-данные, готовит фичи и обучает быстрый top-N для демо. Затем показываем метрики и пробуем локальный инференс без сервера.

Перед запуском укажите ссылку на ваш GitHub-репозиторий в переменной `REPO_URL`.

In [ ]:
#@title Параметры
REPO_URL = "https://github.com/USER/REPO.git"  # <-- ЗАМЕНИТЕ на ваш репозиторий
BRANCH = "main"

import os
import shutil

work = '/content'
repo_dir = os.path.join(work, 'azs_retail_project')
if os.path.exists(repo_dir):
    shutil.rmtree(repo_dir)
print('Cloning repo...')
!git clone -b $BRANCH $REPO_URL $repo_dir
%cd $repo_dir
print('Python version:')
!python --version
print('Upgrade pip and install requirements...')
!pip -q install -U pip setuptools wheel
!pip -q install -r requirements.txt -r requirements-dev.txt


In [ ]:
# Генерация toy-данных и подготовка фич
!python scripts/generate_toy_data.py
!python validation.py
!python etl.py
!python features.py


In [ ]:
# Быстрое обучение per-SKU моделей на маленьком top-N
!python train_forecast.py \
  --train data_raw/train.csv \
  --transactions data_raw/transactions.csv \
  --oil data_raw/oil.csv \
  --holidays data_raw/holidays_events.csv \
  --stores data_raw/stores.csv \
  --models_dir models \
  --warehouse_dir data_dw \
  --top_n_sku 6 \
  --top_recent_days 60 \
  --valid_days 14


In [ ]:
# Просмотр метрик
import pandas as pd

metrics = pd.read_csv('data_dw/metrics_per_sku.csv')
metrics.head(10)


In [ ]:
# Локальный инференс без сервера для одной модели
import glob

import joblib
import numpy as np
import pandas as pd

models = sorted(glob.glob('models/*__*.joblib'))
assert models, 'Нет обученных моделей. Проверьте предыдущую ячейку.'
path = models[0]
print('Using model:', path)
mdl = joblib.load(path)
feat_names = []
if hasattr(mdl, 'feature_name_') and isinstance(mdl.feature_name_, list):
    feat_names = mdl.feature_name_
elif hasattr(mdl, 'feature_names_in_'):
    feat_names = list(mdl.feature_names_in_)
import duckdb

df = duckdb.query("SELECT * FROM read_parquet('data_dw/features.parquet') ORDER BY date DESC LIMIT 1").to_df()
row = df.iloc[0]
X = np.array([float(row.get(n, 0.0)) for n in feat_names], dtype=float).reshape(1, -1)
float(mdl.predict(X)[0])


## Опционально: запуск API и UI (локально в Colab)

Для публичного доступа нужен туннель (ngrok). Для локальной проверки процессов:

```bash
!python -m uvicorn service.app:app --host 0.0.0.0 --port 8000 &
!python -m streamlit run ui/dashboard.py --server.port 8501 --server.address 0.0.0.0 &
```
